1. rdkit: smiles -> mol -> fp (2, 1024)  
2. butina clustering with similarity cutoff  
3. pActivity 값으로 정렬 후 최대값을 반올림 (10.5 -> 11)  
4. 빈 클러스터 리스트 생성  
5. 구간별로 클러스터 번호 확인 -> 위 클러스터 리스트에 없으면 추가 (즉, pActivity 높은 것부터 다양한 클러스터로 선택됨)  
6. 첫 번째 선택 이후에는 클러스터 리스트를 먼저 확인하고 해당 리스트에 있는 클러스터 번호를 가진 물질부터 뽑음 (즉, SAR이 될 수 있게 최상위 active 물질의 클러스터를 가지고 있는 물질을 다음 pActivity 범위에서도 뽑는다.)  
7. 6번 과정을 진행한 다음에는 다시 클러스터 리스트에 없는 물질 선정

In [2]:
import pandas as pd
import rdkit
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit import DataStructs
import os
import math

In [3]:
os.chdir("/Users/siu/MP301_KS_2496_ensemble_result")

In [4]:
fn = 'P53779_BM.csv'
df = pd.read_csv(fn)

In [5]:
col_activity ='pactivity'
col_smiles = 'smiles'

In [6]:
df = df.loc[df[col_activity]!= '-']
df = df.astype({col_activity:'float'})
len(df)

2411

## Butina Clustering

In [7]:
# fps 생성
smiles = df[col_smiles]
ms = [Chem.MolFromSmiles(x) for x in smiles]
fps = [AllChem.GetMorganFingerprintAsBitVect(x,2,1024) for x in ms]

In [8]:
def ClusterFps(fps, cutoff=0.8):
    from rdkit import DataStructs
    from rdkit.ML.Cluster import Butina
    
    distThresh = 1-cutoff
    # first generate the distance matrix:
    dists = []
    nfps = len(fps)
    for i in range(1,nfps):
        sims = DataStructs.BulkTanimotoSimilarity(fps[i],fps[:i])
        dists.extend([1-x for x in sims])

    # now cluster the data:
    cs = Butina.ClusterData(dists,nfps,distThresh,isDistData=True)
    return cs

In [9]:
clusters=ClusterFps(fps,cutoff=0.7)

In [14]:
clusters

((1886,
  304,
  346,
  668,
  863,
  1023,
  1038,
  1060,
  1092,
  1121,
  1836,
  1839,
  1840,
  1842,
  1843,
  1846,
  1847,
  1850,
  1853,
  1855,
  1856,
  1857,
  1858,
  1859,
  1864,
  1867,
  1868,
  1870,
  1871,
  1872,
  1874,
  1877,
  1879,
  1881,
  1882,
  1890,
  1891,
  1892,
  1893,
  1894,
  1895,
  1898,
  1899,
  1900,
  1902,
  1904,
  1905,
  1907,
  1909,
  1911,
  1912,
  1916,
  1917,
  1920,
  1921,
  1923,
  1924,
  1926,
  1929,
  1931,
  1932,
  1935,
  1937,
  1938,
  1939,
  1940,
  1941,
  1942,
  1943,
  1946,
  1947,
  1948,
  1949,
  1955,
  1956,
  1957,
  1958,
  1959,
  1960,
  1961,
  1966,
  1967,
  1970,
  1972,
  1973,
  1975,
  1976,
  1978,
  1980,
  1981,
  1993,
  1994,
  1995,
  1996,
  1999,
  2002,
  2006,
  2007,
  2008,
  2009,
  2011,
  2013,
  2016,
  2020,
  2021,
  2026,
  2032),
 (1394,
  745,
  882,
  896,
  912,
  973,
  980,
  1037,
  1055,
  1068,
  1081,
  1095,
  1137,
  1158,
  1166,
  1184,
  1196,
  1222,
  1234,
 

In [595]:
# 데이터 프레임에 할당해 넣기
df['cl_cluster'] = ''
for i in range(0, len(clusters)) :
    for j in clusters[i]:
        df.loc[j, 'cl_csize'] = int(len(clusters[i]))
        df.loc[j, 'cl_cluster'] = i

## Select 10 Diverses per cluster and pActivity range

In [597]:
number_dv = 10

picked_cluster_all = []
picked_index_all = []

df = df.sort_values(col_activity, ascending=False, ignore_index=True) # pactivty로 정렬
tv = math.ceil(df.loc[0, col_activity]) # pActivity 제일 높은 거
df['orig_idx'] = df.index # 인덱스 백업해두기

for n in range(tv, 0, -1):
    tf = df.loc[(df[col_activity] <= n) & (df[col_activity] > n-1)].sort_values(col_activity, ascending=False).reset_index(drop=True)
    
    picked_cluster_sub = []
    picked_index_sub = []
    
    # 먼저 앞에서 선정된 picked_cluster_all에 있는 것부터 뽑아서 picked_cluster_sub에 추가하기
    # 앞에까지 추가된 picked_cluster_all에 있는 클러스터에 해당하는 top1 물질의 picked_cluster_sub를 먼저 넣음
    # 그냥 클러스터별로 dv N개를 뽑고 싶으면 아래 내용 삭제해도 무방
    
    uniq_clusters = list(set(picked_cluster_all))
    for i in uniq_clusters: # picked_cluster_all에는 중복 있을 수 있음. set 처리 안 하면 중복 발생
        if i in list(tf['cl_cluster']):
            cls = tf.loc[tf['cl_cluster'] == i].reset_index().loc[0, 'cl_cluster'] # 첫번째 것만 추가
            idx = tf.loc[tf['cl_cluster'] == i].reset_index().loc[0, 'orig_idx'] # 첫번째 것만 추가

            picked_index_sub.append(idx)
            picked_cluster_sub.append(cls)
    
    
    # tf 위에서부터 클러스터를 빈 리스트에 추가시키면서, 추가된 것은 스킵하기
    for i in tf.index:
        cls = tf.loc[i, 'cl_cluster']
        idx = tf.loc[i, 'orig_idx']
        if cls not in picked_cluster_sub:
            picked_index_sub.append(idx)
            picked_cluster_sub.append(cls)
    
    # 선택된 것 추가하기
    #picked_cluster_all.append(picked_cluster_sub[:number_dv])
    #picked_index_all.append(picked_index_sub[:number_dv])
    
    # 선택된 것 추가하기 (리스트 평탄화하기 위해 개별 요소별로 추가)
    for i in picked_cluster_sub[:10]:
        picked_cluster_all.append(i)
    
    for i in picked_index_sub[:10]:
        picked_index_all.append(i)

dvdf = df.loc[picked_index_all].reset_index(drop=True)

## Add inactives if it is not enough

In [598]:
# inactives 모자라면 채워넣기, 50-5이하 갯수만큼 랜덤 index로 채워넣기. 없으면 최대한
inactives_cut = 5
inactives_df_raw = df.loc[df[col_activity] < inactive_cut]
inactives_df_sub = dvdf.loc[dvdf[col_activity] < inactive_cut]

In [599]:
short = 50-len(inactives_df_sub) # 50만큼까지 채워야 할 것
num_inactives = len(inactives_df_raw) # 원본에 short만큼 없다면 그만큼만
diff_idx = list(set(inactives_df_raw['orig_idx']) - set(inactives_df_sub['orig_idx'])) # 이미 추가된 idx 제외하고

if len(diff_idx) > 0:
    if num_inactives >= short:
        add_idx = random.sample(diff_idx, short)
    else:
        add_idx = random.sample(diff_idx, num_inactives)
    
    dvdf1 = pd.concat([dvdf, df.loc[add_idx]])
    dvdf1 = dvdf1.sort_values(col_activity, ascending=False).reset_index(drop=True)

else:
    dvdf1 = dvdf

## 중복 확인 및 저장

In [600]:
dvdf1.loc[dvdf1.duplicated(keep=False) == True].sort_values('orig_idx')

,SELECT[Y/N],target,smiles,pactivity,molwt,hba,hbd,rotb,alogp,max_fused_rings,...,source,compound_id,source_smiles,mechanism,drug_info,description,year,cl_cluster,cl_csize,orig_idx


In [601]:
# cluster size 정보를 subset df 정보로 업데이트
dvdf1['cl_csize'] = [list(dvdf1['cl_cluster']).count(x) for x in dvdf1['cl_cluster']] 

In [602]:
len(dvdf1)
dvdf1.to_csv(os.path.splitext(fn)[0]+'_dv.csv')

In [621]:
# Activity / Inactivity 물질 숫자 비율

import glob
ress = glob.glob('*_dv.csv')

for i in ress:
    df = pd.read_csv(i)
    acts = df.loc[df[col_activity] >=7]
    inacts = df.loc[df[col_activity] <5]
    print(i, len(df), len(acts), len(inacts))

P35968_BM_dv.csv 130 59 50
O94768_BM_dv.csv 120 48 50
P08581_BM_dv.csv 107 37 50
Q2M2I8_BM_dv.csv 62 41 6
P17948_BM_dv.csv 110 42 50
Q9NYL2_BM_dv.csv 50 19 5
P36888_BM_dv.csv 120 50 50
P42336_BM_dv.csv 120 50 50
P16234_BM_dv.csv 111 38 50
O00329_BM_dv.csv 116 45 50
Q92630_BM_dv.csv 50 21 5
P00519_BM_dv.csv 120 48 50
P53779_BM_dv.csv 106 32 50
